In [ ]:
import numpy as np
import torch
import torch.nn as nn
from dataset import process_data
from transformer import vit_base_patch16_224_in21k

num_classes = 20
data_path = "dataset"
pre_weights_path = "model/pre_train/jx_vit_base_patch16_224_in21k-e5005f0a.pth"
output_path = "model/output/transformer_zqg.pt"

# 训练轮数
num_epoch = 50
label_list = [
    "南瓜",
    "土豆",
    "小麦",
    "梨子",
    "水稻",
    "玉米",
    "胡萝卜",
    "苹果",
    "茄子",
    "葡萄",
    "蒜苗",
    "西兰花",
    "西瓜",
    "西红柿",
    "豌豆",
    "青椒",
    "韭菜",
    "香菜",
    "香蕉",
    "黄瓜",
]


def show_predictResult(pre_label, truth_label):
    for pre, truth in zip(pre_label, truth_label):
        print(f"预测类别为{label_list[int(pre)]}, 而真实类别为{label_list[int(truth)]}")


def show_progress(i, length):
    progress = (i + 1) / length  # 进度条的完成比例
    bar_width = 100
    num_blocks = int(progress * bar_width)
    progress_bar = "\u2588" * num_blocks + " " * (bar_width - num_blocks)
    print(f"Progress: [{progress_bar}] {progress*100:.6f}%")

In [ ]:
#创建vit模型
model = vit_base_patch16_224_in21k(num_classes=num_classes, has_logits=False)

print("start get data...")
train_loader, train_label, label_list = process_data(data_path + "/train")
print("already process data...")

print("start prepared model...")
print("if GPU can be used?:", torch.cuda.is_available())  # true 查看GPU是否可用
model.cuda()  # 利用GPU加速
# 加载预训练权重参数
weights = torch.load(pre_weights_path)
# 应用权重参数到模型实例
model.load_state_dict(weights)
print("already prepared model...")

In [ ]:
print("start training...")
loss = nn.CrossEntropyLoss()  # 定义损失函数
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # 定义优化器
for epoch in range(num_epoch):
    train_loss = 0
    train_acc = 0
    model.train()  # 模型置于训练状态
    for i, data in enumerate(train_loader):  # train_loader = (inputs, label)
        inputs = data[0].type(torch.FloatTensor)
        label = data[1].type(torch.FloatTensor)

        optimizer.zero_grad()  # 梯度清零
        pred_label = model(inputs.cuda())  # 预测的输出
        batch_loss = loss(pred_label, label.cuda().long())  # 计算误差
        batch_loss.backward()  # 反向传播
        optimizer.step()  # 更新参数

        train_acc += np.sum(
            np.argmax(pred_label.cpu().data.numpy(), axis=1) == label.numpy()
        )  # 计算准确度
        
        # train_acc += (torch.argmax(pred_label, dim=1) == label.cuda()).sum().item()
        show_predictResult(
            np.argmax(pred_label.cpu().data.numpy(), axis=1),
            label.numpy(),
        )
        show_progress(i, len(train_loader))  #显示每一轮的训练进度

        train_loss += batch_loss.item()  # 计算误差

    print("epoch:{}".format(epoch + 1))
    print("model's accuracy in train_data is:{}".format(train_acc / len(train_label)))
    print("model's loss in train_data is:{}".format(train_loss / len(train_label)))
    print(f"开始保存第{epoch}轮的训练结果")
    torch.save(model.state_dict(), output_path)  # 导出.pt文件
    print(f"第{epoch}轮的训练结果已保存")
    if train_acc / len(train_label) > 0.98:
        break
print("finish training...")

print("start testing...")
test_loader, test_label = process_data(data_path + "/test")
model.eval()  # 将模型置于评估状态
test_acc = 0
with torch.no_grad():
    for i, data in enumerate(test_loader):
        data[0] = data[0].type(torch.FloatTensor)
        data[1] = data[1].type(torch.FloatTensor)
        pred_label = model(data[0].cuda())  # 预测输出
        test_acc += np.sum(
            np.argmax(pred_label.cpu().data.numpy(), axis=1) == data[1].numpy()
        )  # 计算准确度
    print("model's accuracy in test_data is:{}".format(test_acc / len(test_label)))
print("finish testing...")